<a href="https://colab.research.google.com/github/jess789550/machinelearning/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import os

#!pip install -q tflite-model-maker  # ERROR
#from tflite_model_maker.text_classifier import DataLoader


print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# My code using Google and https://www.tensorflow.org/tutorials/keras/text_classification
# https://ai.google.dev/edge/litert/libraries/modify/text_classification
with open(train_file_path) as f:
  print(f.read())

with open(test_file_path) as f:
  print(f.read())

# CANNOT INSTALL DataLoader
# train_data = DataLoader.from_csv(
#       filename='train.csv',
#       text_column='sentence',
#       label_column='label',
#       model_spec=spec,
#       is_training=True)

# test_data = DataLoader.from_csv(
#       filename='dev.csv',
#       text_column='sentence',
#       label_column='label',
#       model_spec=spec,
#       is_training=False)

batch_size = 32
seed = 42

# NOT DIRECTORY
# raw_train_ds = tf.keras.utils.text_dataset_from_directory(
#     train_file_path,
#     batch_size=batch_size,
#     validation_split=0.2,
#     subset='training',
#     seed=seed)

In [ ]:
# ChatGPT

import pandas as pd

# Load the training and testing data without headers, then assign column names
train_data = pd.read_csv("train-data.tsv", sep='\t', header=None, names=["label", "message"])
test_data = pd.read_csv("valid-data.tsv", sep='\t', header=None, names=["label", "message"])

# Check the first few rows to ensure it loaded correctly
train_data.head(), test_data.head()

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report

# Clean and preprocess the text
def preprocess_text(text):
    text = text.lower()  # Convert text to lowercase
    # Additional preprocessing can be done here, like removing punctuation, etc.
    return text

# Apply preprocessing to train and test data
train_data['cleaned_text'] = train_data['message'].apply(preprocess_text)
test_data['cleaned_text'] = test_data['message'].apply(preprocess_text)

# Create a simple pipeline with a CountVectorizer and Naive Bayes classifier
model = make_pipeline(CountVectorizer(), MultinomialNB())

# Split the data into features and labels
X_train = train_data['cleaned_text']
y_train = train_data['label']

# Train the model
model.fit(X_train, y_train)

# Evaluate the model on the validation set
X_test = test_data['cleaned_text']
y_test = test_data['label']
y_pred = model.predict(X_test)

# Print evaluation metrics
print(classification_report(y_test, y_pred))

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  message = preprocess_text(pred_text) # ChatGPT

  # Predict the class using the trained model
  prob = model.predict_proba([message])  # Get the probabilities for each class
  ham_prob, spam_prob = prob[0]  # prob[0] gives the probabilities for the two classes

  prediction_num = spam_prob

  if spam_prob > ham_prob:
    label = "spam"
  else:
    label = "ham"

  prediction = [prediction_num, label]
  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
